In [1]:
import pandas as pd
import numpy as np
from sklearn import datasets as datasets
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


In [2]:
house = datasets.load_boston()
bcancer = datasets.load_breast_cancer()

In [14]:
# Housing data target variable is price: continuous!
# If Y is continuous, use regression. Regression predicts mean Y (E[Y] given X)
# alternatively Y ~ B*X
Xh, Yh = pd.DataFrame(house.data, columns=house.feature_names), house.target

# Breast cancer target is categorical; has breast cancer or not.
# If Y is categorical, use classification. 
# Classification predicts probability of category (E[P(Y == 1)])
# alternatively (for logistic regression only though): Y ~ BX
Xb, Yb = pd.DataFrame(bcancer.data, columns=bcancer.feature_names), bcancer.target

In [10]:
print Xh.columns
print Xb.columns

Index([u'CRIM', u'ZN', u'INDUS', u'CHAS', u'NOX', u'RM', u'AGE', u'DIS',
       u'RAD', u'TAX', u'PTRATIO', u'B', u'LSTAT'],
      dtype='object')
Index([u'mean radius', u'mean texture', u'mean perimeter', u'mean area',
       u'mean smoothness', u'mean compactness', u'mean concavity',
       u'mean concave points', u'mean symmetry', u'mean fractal dimension',
       u'radius error', u'texture error', u'perimeter error', u'area error',
       u'smoothness error', u'compactness error', u'concavity error',
       u'concave points error', u'symmetry error', u'fractal dimension error',
       u'worst radius', u'worst texture', u'worst perimeter', u'worst area',
       u'worst smoothness', u'worst compactness', u'worst concavity',
       u'worst concave points', u'worst symmetry', u'worst fractal dimension'],
      dtype='object')


In [12]:
np.unique(Yb)

array([0, 1])

In [13]:
Yh[0:10]

array([ 24. ,  21.6,  34.7,  33.4,  36.2,  28.7,  22.9,  27.1,  16.5,  18.9])

### LINEAR REGRESSION

In [19]:
# check predictor types in case there are categorical in there...
Xh.dtypes

# IF  YOU HAVE LOTS OF CATEGORICAL, USING PATSY IS WAY LESS WORK

CRIM       float64
ZN         float64
INDUS      float64
CHAS       float64
NOX        float64
RM         float64
AGE        float64
DIS        float64
RAD        float64
TAX        float64
PTRATIO    float64
B          float64
LSTAT      float64
dtype: object

In [15]:
from sklearn.linear_model import LinearRegression

In [81]:
# First construct the linear regression using the "blueprint"
linear_reg = LinearRegression()

# Fit the linear regression on your target and predictors:
# x is pandas dataframe, so convert to matrix! keep track of column names
Xh_columns = Xh.columns
Xh_mat = Xh.values

house_linreg = linear_reg.fit(Xh_mat, Yh)

In [82]:
# PREDICTIONS!
Yh_predictions = house_linreg.predict(Xh_mat)

print Yh[0:10]
print Yh_predictions[0:10]

[ 24.   21.6  34.7  33.4  36.2  28.7  22.9  27.1  16.5  18.9]
[ 30.00821269  25.0298606   30.5702317   28.60814055  27.94288232
  25.25940048  23.00433994  19.5347558   11.51696539  18.91981483]


In [83]:
# Is our model good? Let's check the R2 for the model.
# Since we are just using our original data to predict, put it in the score function.
house_linreg.score(Xh_mat, Yh)

# WTF is R2?
# this is the proportion of variance explained in our target variable 
# COMPARED TO A MODEL THAT JUST USES MEAN OF Y (baseline model)
# aka: how much better is the model compared to just guessing every Y row with the mean
# of Y.

0.74060774286494269

In [24]:
# what are our coefficients from the model?
print house_linreg.coef_

[ -1.07170557e-01   4.63952195e-02   2.08602395e-02   2.68856140e+00
  -1.77957587e+01   3.80475246e+00   7.51061703e-04  -1.47575880e+00
   3.05655038e-01  -1.23293463e-02  -9.53463555e-01   9.39251272e-03
  -5.25466633e-01]


In [25]:
# look at this in a nicer way. we saved the column values earlier, conveniently
house_coefs = pd.DataFrame({'feature':Xh_columns, 'coef':house_linreg.coef_})

# Remember that for linear regression the formula is Y ~ b1*x1 + ... + bn*xn
# for our x1 thru xn columns of predictors.
# To estimate Y for a row of predictor variables, we multiply each of these
# coefficients by their respective beta coefficients and add them together!
house_coefs

,coef,feature
0,-0.107171,CRIM
1,0.046395,ZN
2,0.020860,INDUS
3,2.688561,CHAS
4,-17.795759,NOX
5,3.804752,RM
6,0.000751,AGE
7,-1.475759,DIS
8,0.305655,RAD
9,-0.012329,TAX


### LASSO

Why would we use the Lasso?

Lasso, depending on the **regularization strength alpha**, will eliminate variables in order of their value or importance on predicting Y.

In this case we probably don't need it for the data (will likely make prediction worse if we remove variables), but this is just a demonstration.

In [26]:
from sklearn.linear_model import Lasso

In [28]:
# initialize Lasso just as you would the linear regression...

# Let's make 2. One with "weak" regularization, which means there is a
# very small penalty on coefficient sizes.
# Coefficients added up can be big and it won't really care (basically
# will be the same as vanilla Linear Regression)
house_lasso_weak = Lasso(alpha=0.01)

# make one that has strong regularization
house_lasso_strong = Lasso(alpha=100.)

In [30]:
# fit them each on data
h_weak = house_lasso_weak.fit(Xh_mat, Yh)
h_strong = house_lasso_strong.fit(Xh_mat, Yh)

In [31]:
# make tables of coefficients to see what lassos did
weak_coefs = pd.DataFrame({'feature':Xh_columns, 'coef':h_weak.coef_})
strong_coefs = pd.DataFrame({'feature':Xh_columns, 'coef':h_strong.coef_})


In [32]:
# what does weak look like?
weak_coefs

,coef,feature
0,-0.105331,CRIM
1,0.046833,ZN
2,0.006776,INDUS
3,2.506079,CHAS
4,-14.422629,NOX
5,3.809185,RM
6,-0.001761,AGE
7,-1.422294,DIS
8,0.298100,RAD
9,-0.012622,TAX


In [33]:
strong_coefs

,coef,feature
0,-0.000000,CRIM
1,0.000000,ZN
2,-0.000000,INDUS
3,0.000000,CHAS
4,-0.000000,NOX
5,0.000000,RM
6,-0.000000,AGE
7,0.000000,DIS
8,-0.000000,RAD
9,-0.020972,TAX


In [34]:
# When fit on its own data, it will be worse than LinearRegression
# but when using crossvalidation it will sometimes perform better.
house_lasso_strong.score(Xh_mat, Yh)

0.22497922550751603

### RIDGE REGRESSION

Ridge penalty has a different effect on our coefficients. Without getting into the math (you can review the slides in the lesson), it "shrinks" all our coefficients down.

This is particularly useful if a lot of our predictor variables are correlated with each other.

Why?

Basically, because the coefficients shrink for the correlated variables, they can contribute more equally to the prediction. The reason that can be good is that **in our sample of data, one may be a better predictor than the other, but in a new sample, the other one might be better!**

You are hedging your bets on the predictors.

In [35]:
from sklearn.linear_model import Ridge

In [36]:
# Again, set up two ridges, one with strong regularization, one with weak.
# Weak will be similar if not the same to the LinearRegression
# Strong will make the beta values very small, but more equal to each other in magnitude.
h_ridge_weak = Ridge(alpha=0.01)
h_ridge_strong = Ridge(alpha=100)

In [40]:
# fit models
ridge_weak_mod = h_ridge_weak.fit(Xh_mat, Yh)
ridge_strong_mod = h_ridge_strong.fit(Xh_mat, Yh)

In [41]:
ridge_weak_coefs = pd.DataFrame({'feature':Xh_columns, 'coef':ridge_weak_mod.coef_})
ridge_strong_coefs = pd.DataFrame({'feature':Xh_columns, 'coef':ridge_strong_mod.coef_})


In [42]:
ridge_weak_coefs

,coef,feature
0,-0.107111,CRIM
1,0.046411,ZN
2,0.020377,INDUS
3,2.686838,CHAS
4,-17.681068,NOX
5,3.805660,RM
6,0.000649,AGE
7,-1.474067,DIS
8,0.305385,RAD
9,-0.012338,TAX


In [43]:
ridge_strong_coefs

,coef,feature
0,-0.101451,CRIM
1,0.054470,ZN
2,-0.052626,INDUS
3,0.638647,CHAS
4,-0.263245,NOX
5,2.331966,RM
6,0.001230,AGE
7,-1.153157,DIS
8,0.314915,RAD
9,-0.015852,TAX


# CLASSIFICATION!

### K Nearest Neighbors

In [44]:
from sklearn.neighbors import KNeighborsClassifier

In [47]:
# kNN classifies between categories (in the case of the breast cancer data, 
# it is going to classify observations (X rows) as either 1 or 0).

# The way it does this is by checking out the "neighbors" of an observation.
# The nearest neighbor of an observation is the one that has the most 
# similar values for each X predictor. It calculates (typically)
# the euclidean distance.

# You choose how many neighbors will "take a vote" on what the class
# of the new observation should be. The voting can be equal
# (weights='uniform') which means they all get an equal vote, or
# the weighting can be adjusted for distance (weights='distance')
# where the closer you are, the more important your vote is.

knn_3neighbors = KNeighborsClassifier(n_neighbors=3)
knn_11neighbors = KNeighborsClassifier(n_neighbors=11)



In [50]:
Xb_mat = Xb.values
Xb_cols = Xb.columns

# fit the x, y data for the neighbors:
knn3 = knn_3neighbors.fit(Xb_mat, Yb)
knn11 = knn_11neighbors.fit(Xb_mat, Yb)

In [51]:
# score the models - the default score for classifiers is the accuracy
# this is the proportion of observations that were assigned the correct
# label.

# First look at the "base rate" accuracy, which is the proportion of 1s 
# relative to zeros.
print np.mean(Yb)

print 'knn3 acc:', knn3.score(Xb_mat, Yb)
print 'knn11 acc:', knn11.score(Xb_mat, Yb)

0.627416520211
knn3 acc: 0.956063268893
knn11 acc: 0.940246045694


### kNN bias-variance in a nutshell

**Smaller number of neighbors = higher variance, lower bias**: 

**Higher number of neighbors = lower variance, higher bias:** in the extreme, if you use all the neighbors (all the points), you are just estimating the mean number of Y 1s relative to 0s (note: this is only in the case of uniform weights, but bias increases for distance too, just not to the mean.)


### Logistic regression with Ridge and Lasso

We can use ridge and lasso in logistic regression just like with normal regression problems, but logistic regression is predicting probabilities not mean values!

**NOTE: the regularization parameter is now C instead of alpha, which is basically 1/alpha (inverse of regularization strength).**




In [52]:
from sklearn.linear_model import LogisticRegression

In [72]:
# set up the standard, ridge, and lasso logistic regressions:
lr_standard = LogisticRegression()
# REMEMBER: the penalty string is lowercase "L" first, NOT 1
lr_lasso = LogisticRegression(penalty='l1', solver='liblinear', C=0.1)
lr_ridge = LogisticRegression(penalty='l2', C=0.001)

In [73]:
# NORMALIZE THE PREDICTOR VARIABLES!
# this makes the betas relative to each other, so you can directly compare the
# effect of one variable on the log odds to another variable. Otherwise they
# are very difficult to interpret
Xb_mat = ((Xb - Xb.mean()) / Xb.std()).values

In [74]:
lr_std_mod = lr_standard.fit(Xb_mat, Yb)
lr_l1_mod = lr_lasso.fit(Xb_mat, Yb)
lr_l2_mod = lr_ridge.fit(Xb_mat, Yb)

In [75]:
# get out the coefficients for each model so you can see the effect of ridge, lasso 
# on the logistic regression coefficients.
#
# logistic regression coefficients are a list of lists. This is because in the
# multinomial (many class) case it will have the coefficients for each logistic
# regression that was fit. In the binary case, there is just one list inside.
def make_coef_df(Xb, mod):
    df = pd.DataFrame({'features':Xb.columns, 'coefs':mod.coef_[0]})
    return df

lr_std_coefs = make_coef_df(Xb, lr_std_mod)
lr_l1_coefs = make_coef_df(Xb, lr_l1_mod)
lr_l2_coefs = make_coef_df(Xb, lr_l2_mod)


In [76]:
# look at standard logistic regression coefficients
lr_std_coefs

,coefs,features
0,-0.354258,mean radius
1,-0.385699,mean texture
2,-0.342896,mean perimeter
3,-0.441885,mean area
4,-0.155334,mean smoothness
5,0.567933,mean compactness
6,-0.868805,mean concavity
7,-0.968382,mean concave points
8,0.073177,mean symmetry
9,0.311547,mean fractal dimension


In [77]:
# look at ridge coefs:
lr_l2_coefs

,coefs,features
0,-0.087617,mean radius
1,-0.056655,mean texture
2,-0.088180,mean perimeter
3,-0.083912,mean area
4,-0.037799,mean smoothness
5,-0.058437,mean compactness
6,-0.074833,mean concavity
7,-0.089509,mean concave points
8,-0.033346,mean symmetry
9,0.015474,mean fractal dimension


In [78]:
lr_l1_coefs

,coefs,features
0,0.000000,mean radius
1,0.000000,mean texture
2,0.000000,mean perimeter
3,0.000000,mean area
4,0.000000,mean smoothness
5,0.000000,mean compactness
6,0.000000,mean concavity
7,-0.629997,mean concave points
8,0.000000,mean symmetry
9,0.000000,mean fractal dimension


# MODEL EVALUATION

---

### Regression model metric $R^2$

In [84]:
# Regression typically uses R2 as its metric.
# R2 is the amount of variance in Y explained by your model over the variance 
# explained by the mean model, or intercept model.

# Get the R2 (on the data we trained on... R2 is in this case an "inflated"
# estimation of how much variance we will explain for new data. This is
# why we use cross-validation.)
print house_linreg.score(Xh_mat, Yh)

# 1 is the maximum for R2
# 0 is the minimum for R2 (when testing on the training data)

# NOTE: you may have seen R2 that are negative..
# This can happend when you test your model on held-out data
# What does a negative R2 mean?
# This means the model is worse at predicting Y than just using the mean of Y.

0.740607742865


### Classification metrics:

Classification metrics are more various. This is because we can have priorities about how many true positives, false positives we want. Review the classification metrics lecture for this to review in depth.

- Accuracy if we just want to get the best proportion of correct guesses to incorrect guesses.
- If you don't many false negatives (as in telling someone they have cancer or not), then accuracy is probably not what you want to optimize for.

In [85]:
# fit normally and calculate the accuracy
lr_std_mod = lr_standard.fit(Xb_mat, Yb)
print lr_std_mod.score(Xb_mat, Yb)

0.98769771529


In [86]:
# cross-validate the accuracy:
# CROSS-VALIDATION
# if we test on the data we trained on, then we are not getting a good sense
# of how well it will perform on new data.
# Cross-validation splits up the data you have, it trains on a fraction of
# the data, then scores it on the data you held out.
#
# Why is this better?
# Your model will necessarily perform the best it can on your own data, because
# the model literally optimizedd the predictor coefficients to get as close
# to the target values as possible.
# If your current data is not representative of new data, it is likely to do
# poorly. SO your estimation of performance is inflated.
#
# With cross-validation, you are EMULATING having new data, by leavinng out
# data as if it were new, then scoring on that.
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(lr_std_mod, Xb_mat, Yb, cv=5)

print scores
print np.mean(scores)

[ 0.9826087   0.97391304  0.97345133  0.97345133  0.99115044]
0.978914967295


In [94]:
# Print the confusion matrix, TP, TN, FP, FN
y_pred = lr_std_mod.predict(Xb_mat)
print y_pred.shape, Yb.shape

pd.crosstab(Yb, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

(569,) (569,)


Predicted,0,1,All
True,,,
0,207,5,212
1,2,355,357
All,209,360,569


In [88]:
# Now we want to have no false negatives. We don't want to tell someone they don't
# have cancer when they actually do!

# How to do this?
# First get the model's predicted probabilities of each class.
predicted_probabilities = lr_std_mod.predict_proba(Xb_mat)

In [89]:
print predicted_probabilities[0:5]

[[  9.99999999e-01   1.03870491e-09]
 [  9.99970076e-01   2.99244560e-05]
 [  9.99999845e-01   1.55415263e-07]
 [  9.99538770e-01   4.61230056e-04]
 [  9.99972364e-01   2.76357028e-05]]


In [90]:
Yb[0:5]

array([0, 0, 0, 0, 0])

In [93]:
# change the threshold for deciding whether something is 0:
# we want to be 95 percent confident a person doesnt have cancer to say they dont,
# instead of the standard if over 50% confident they don't say they dont.
y_new_pred = [0 if pp[0] > 0.95 else 1 for pp in predicted_probabilities]
print len(Yb), len(y_new_pred)

569 569


In [95]:
pd.crosstab(np.array(Yb), np.array(y_new_pred),
            rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,0,1,All
True,,,
0,181,31,212
1,0,357,357
All,181,388,569
